In [1]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np

In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.

1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  
a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  
b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.  
c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  
d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [3]:
URL = 'https://realpython.github.io/fake-jobs/'
response = requests.get(URL)
soup = BS(response.text)

In [4]:
first_job = soup.find('h2').text
job_titles = soup.findAll('h2', attrs = {'class' : 'title is-5'})
jobs_list = [tag.text for tag in job_titles]

In [5]:
companies = soup.findAll('h3', attrs = {'class' : 'subtitle is-6 company'})
company_list = [tag.text for tag in companies]

In [6]:
locations = soup.findAll('p', attrs = {'class' : 'location'})
location_list = [tag.text for tag in locations]

In [7]:
post_date = soup.findAll('p', attrs = {'class' : 'is-small has-text-grey'})
post_dates = [tag.text for tag in post_date]

In [8]:
data = [jobs_list,company_list,location_list,post_dates]
scraps = pd.DataFrame(data).transpose()
scraps.columns = ['jobs', 'company', 'location', 'date_posted']
scraps

,jobs,company,location,date_posted
0,Senior Python Developer,"Payne, Roberts and Davis","\n Stewartbury, AA\n",\n2021-04-08\n
1,Energy engineer,Vasquez-Davidson,"\n Christopherville, AA\n",\n2021-04-08\n
2,Legal executive,"Jackson, Chambers and Levy","\n Port Ericaburgh, AA\n",\n2021-04-08\n
3,Fitness centre manager,Savage-Bradley,"\n East Seanview, AP\n",\n2021-04-08\n
4,Product manager,Ramirez Inc,"\n North Jamieview, AP\n",\n2021-04-08\n
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","\n Lake Abigail, AE\n",\n2021-04-08\n
96,"Radiographer, diagnostic",Holder LLC,"\n Jacobshire, AP\n",\n2021-04-08\n
97,Database administrator,Yates-Ferguson,"\n Port Susan, AE\n",\n2021-04-08\n
98,Furniture designer,Ortega-Lawrence,"\n North Tiffany, AA\n",\n2021-04-08\n


In [9]:
scraps['location'] = scraps['location'].str.extract(r'\s(\w\D+,\s\D{2})')
scraps['date_posted'] = scraps['date_posted'].str.extract(r'(\d{4}-\d{2}-\d{2})')
scraps

,jobs,company,location,date_posted
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.   
    a. First, use the BeautifulSoup find_all method to extract the urls.  
    b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [11]:
#apply_list
apply = soup.findAll('a', string = "Apply")

In [12]:
apply_links = []

for link in apply:
    apply_links.append(link.get('href'))

In [13]:
scraps['url'] = scraps['jobs'].astype(str).str.lower()

In [14]:
dictionary = {'(':'',
              ')':'',
              ',':'',
              '/':'',
              ' ':'-'} 
for key, value in dictionary.items():
     scraps['url'] = scraps['url'].str.replace(key, value).str.lower()

In [15]:
for index, x in enumerate(scraps['url']):
    scraps.loc[index, 'index'] = index

In [16]:
scraps['index'] = scraps['index'].astype(int).astype(str)
scraps.head()

,jobs,company,location,date_posted,url,index
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,senior-python-developer,0
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,energy-engineer,1
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,legal-executive,2
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,fitness-centre-manager,3
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,product-manager,4


In [17]:
scraps['url_link'] = "https://realpython.github.io/fake-jobs/jobs/" + scraps['url'] + "-" + scraps['index'] + ".html"

    3. Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  
    b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  
    c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [19]:
URL2 = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'
response2 = requests.get(URL2)
soup2 = BS(response2.text)

In [20]:
def job_description(URL_input):
     print(BS(requests.get(URL_input).text).findAll('p')[1].text)

In [21]:
#didn't work well
#desc_list = []
#desc_list.append(scraps['url_link'].apply(job_description))

In [22]:
link_list = scraps['url_link'].tolist()

In [23]:
description_list = []
for url in link_list:
    description_list.append(BS((requests.get(url)).text).findAll('p')[1].text)

In [24]:
scraps['descriptions'] = description_list
scraps['descriptions'] = scraps['descriptions'].str.strip()
scraps['descriptions']

0     Professional asset web application environment...
1     Party prevent live. Quickly candidate change a...
2     Administration even relate head color. Staff b...
3     Tv program actually race tonight themselves tr...
4     Traditional page a although for study anyone. ...
                            ...                        
95    The site configured at this address does not\n...
96    Able such right culture. Wrong pick structure ...
97    Create day party decade high clear. Past trade...
98    Pressure under rock next week. Recognize so re...
99    Management common popular project only. Must s...
Name: descriptions, Length: 100, dtype: object

In [25]:
scraps.head()

,jobs,company,location,date_posted,url,index,url_link,descriptions
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,senior-python-developer,0,https://realpython.github.io/fake-jobs/jobs/se...,Professional asset web application environment...
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,energy-engineer,1,https://realpython.github.io/fake-jobs/jobs/en...,Party prevent live. Quickly candidate change a...
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,legal-executive,2,https://realpython.github.io/fake-jobs/jobs/le...,Administration even relate head color. Staff b...
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,fitness-centre-manager,3,https://realpython.github.io/fake-jobs/jobs/fi...,Tv program actually race tonight themselves tr...
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,product-manager,4,https://realpython.github.io/fake-jobs/jobs/pr...,Traditional page a although for study anyone. ...
